## 3. Hardware Build <a id='vivado'></a>

We're finally ready to start generating hardware from our network. Depending on whether you want to target a Zynq or Alveo platform, FINN offers two transformations to build the accelerator, integrate into an appropriate shell and build a bitfile. These are `ZynqBuild` and `VitisBuild` for Zynq and Alveo, respectively. In this notebook we'll demonstrate the `ZynqBuild` as these boards are more common and it's much faster to complete bitfile generation for the smaller FPGAs found on them.

In previous versions of FINN, we had to manually go through several steps to generate HLS/RTL code, stitch IP, create a PYNQ project and run synthesis. All these steps are now performed by the `ZynqBuild` transform (or the `VitisBuild` transform for Alveo). **As this involves calling HLS synthesis and Vivado synthesis, this transformation will run for some time (up to half an hour depending on your PC).**

In [2]:
build_dir = "./"

In [4]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from qonnx.core.modelwrapper import ModelWrapper


# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

model = ModelWrapper(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

dict_keys(['Ultra96', 'Ultra96-V2', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104', 'ZCU111', 'RFSoC2x2', 'RFSoC4x2', 'KV260_SOM'])


/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 10 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:237: UserWarning: Input FIFO for IODMA_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:299: UserWarning: Output FIFO for LabelSelect_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/src/finn/transformation/fpgadataflow/create_stitched_ip.py:290: User

After the `ZynqBuild` we run one additional transformation to generate a PYNQ driver for the accelerator.

In [5]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))

In [6]:
model.save(build_dir + "/tfc_w1_a1_post_synthesis.onnx")

### Examining the generated outputs <a id='gen_outputs'></a>

Let's start by viewing the post-synthesis model in Netron:

In [3]:
from finn.util.visualization import showSrc, showInNetron

showInNetron(build_dir + "/tfc_w1_a1_post_synthesis.onnx", port=2222)

Serving './/tfc_w1_a1_post_synthesis.onnx' at http://0.0.0.0:2222


We can see that our sequence of HLS layers has been replaced with `StreamingDataflowPartition`s, each of which point to a different ONNX file. You can open a Netron session for each of them to view their contents. Here, the first and last partitions contain only an `IODMA` node, which was inserted automatically to move data between DRAM and the accelerator. Let's take a closer look at the middle partition, which contains all our layers:

In [7]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.custom_op.registry import getCustomOp

model = ModelWrapper(build_dir + "/tfc_w1_a1_post_synthesis.onnx")
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")

showInNetron(postsynth_layers, port=2222)

Serving '/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//dataflow_partition_ntxhcs80/partition_2.onnx' at http://0.0.0.0:2222


We can see that `StreamingFIFO` and `StreamingDataWidthConverter` instances have been automatically inserted into the graph prior to hardware build. Both layer types are inserted as RTL variants. Transformations like `ZynqBuild` use the `metadata_props` of the model to put in additional metadata information relevant to the results of the transformation. Let's examine the metadata for the current graph containing all layers:

In [8]:
model = ModelWrapper(postsynth_layers)
model.model.metadata_props

[key: "floorplan_json"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vitis_floorplan_i57skffa/floorplan.json"
, key: "vivado_stitch_proj"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_stitch_proj_4nctmwdy"
, key: "clk_ns"
value: "10"
, key: "wrapper_filename"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_stitch_proj_4nctmwdy/finn_vivado_stitch_proj.gen/sources_1/bd/StreamingDataflowPartition_1/hdl/StreamingDataflowPartition_1_wrapper.v"
, key: "vivado_stitch_vlnv"
value: "xilinx_finn:finn:StreamingDataflowPartition_1:1.0"
, key: "vivado_stitch_ifnames"
value: "{\"clk\": [\"ap_clk\"], \"rst\": [\"ap_rst_n\"], \"s_axis\": [[\"s_axis_0\", 392]], \"m_axis\": [[\"m_axis_0\", 8]], \"aximm\": [], \"axilite\": []}"
, key: "platform"
value: "zyn

Here we see that a Vivado project was built to create what we call the `stitched IP`, where all the IP blocks implementing various layers will be stitched together. You can view this stitched block design in Vivado, or [here](StreamingDataflowPartition_1.pdf) as an exported PDF.

Moving back to the top-level model, recall that `ZynqBuild` will create a Vivado project and synthesize it, so it will be creating metadata entries related to the paths and files that were created:

In [9]:
model = ModelWrapper(build_dir + "/tfc_w1_a1_post_synthesis.onnx")
model.model.metadata_props

[key: "floorplan_json"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vitis_floorplan_i57skffa/floorplan.json"
, key: "vivado_pynq_proj"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_zynq_proj_4lzsijk8"
, key: "bitfile"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_zynq_proj_4lzsijk8/resizer.bit"
, key: "hw_handoff"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_zynq_proj_4lzsijk8/resizer.hwh"
, key: "vivado_synth_rpt"
value: "/home/surajkarki/Documents/My_Workspace/Coding_Works/Master_Project/finn/notebooks/experiments/yolov8/finn_build_dir//vivado_zynq_proj_4lzsijk8/synth_report.xml"
, key: "platform"
value: "zynq-iodma"
, key: "pynq_driver_dir"
value

Here, we can see the directories that were created for the PYNQ driver (`pynq_driver_dir`) and the Vivado synthesis project (`vivado_pynq_proj`), as well as the locations of the bitfile, hardware handoff file and synthesis report.

In [10]:
! ls {model.get_metadata_prop("vivado_pynq_proj")}

finn_zynq_link.cache	      finn_zynq_link.srcs  resizer.hwh
finn_zynq_link.gen	      finn_zynq_link.xpr   synth_project.sh
finn_zynq_link.hw	      ip_config.tcl	   synth_report.xml
finn_zynq_link.ip_user_files  NA		   vivado.jou
finn_zynq_link.runs	      resizer.bit	   vivado.log


Feel free to examine the generated Vivado project to get a feel for how the system-level integration is performed for the  FINN-generated "stitched IP", which appears as `StreamingDataflowPartition_1` in the top-level block design -- you can see it as a block diagram exported to PDF [here](top.pdf).